<a href="https://colab.research.google.com/github/dayananikol/CCADMACL_EXERCISES_COM222ML/blob/main/advml_exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 1

Use all feature selection methods to find the best features

## Dataset Information

## Features

Number of Instances: 20640

Number of Attributes: 8 numeric, predictive attributes and the target

Attribute Information:

MedInc - median income in block group

HouseAge - median house age in block group

AveRooms - average number of rooms per household

AveBedrms - average number of bedrooms per household

Population - block group population

AveOccup - average number of household members

Latitude - block group latitude

Longitude - block group longitude

## Target
The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

In [1]:
from sklearn.datasets import fetch_california_housing

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
housing = fetch_california_housing(as_frame=True)
df = pd.concat([housing.data, housing.target], axis=1)

In [3]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [5]:
score_list = {}

In [6]:
X = housing.data
y = housing.target

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,test_size=0.3)

In [7]:
from sklearn.ensemble import RandomForestRegressor

rf_model0 = RandomForestRegressor(n_estimators=100, max_depth=4, random_state=42)
rf_model0.fit(X_train, y_train)
accuracy_score_rf0 = rf_model0.score(X_test, y_test)
accuracy_score_rf0

0.6004189703723741

In [8]:
score_list["W/O Feature Selection"] = accuracy_score_rf0

1. Use any filter method to select the best features

In [9]:
selector = VarianceThreshold(threshold=0.5) # Variance threshold
sel = selector.fit(housing.data)
sel_index = sel.get_support()
df_norm_vt = housing.data.iloc[:, sel_index]

In [10]:
df_norm_vt

,MedInc,HouseAge,AveRooms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,741.0,2.123209,39.43,-121.32


In [11]:
filterX = df_norm_vt
filtery=df['MedHouseVal']

X_train1,X_test1,y_train1,y_test1 = train_test_split(filterX,filtery,random_state=0,test_size=0.3)

In [12]:
rf_model1 = RandomForestRegressor(n_estimators=500, max_depth=3, random_state=0)
rf_model1.fit(X_train1, y_train1)
accuracy_score_rf1 = rf_model1.score(X_test1, y_test1)
accuracy_score_rf1

0.542014790297386

In [13]:
score_list["Filter Method"] = accuracy_score_rf1

2. Use any wrapper method to select the best features

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

threshold = 5 # the number of most relevant features
model_rf2 = RandomForestRegressor(n_estimators=500, max_depth=3, random_state=0)
selector = RFE(model_rf2, n_features_to_select=5, step=1)

wrapperX = df.drop('MedHouseVal', axis=1) # Features
wrappery = df['MedHouseVal'] # Target variable


selector = selector.fit(wrapperX, wrappery)
selector_ind = selector.get_support()
df_rfe = wrapperX.iloc[:, selector_ind]
df_rfe.columns

X_train2,X_test2,y_train2,y_test2 = train_test_split(df_rfe,wrappery,random_state=0,test_size=0.3)

In [15]:
from sklearn.ensemble import RandomForestRegressor

model_rf2.fit(X_train2, y_train2)
accuracy_score_rf2 = model_rf2.score(X_test2, y_test2)
accuracy_score_rf2

0.5420111606499833

In [16]:
score_list["Wrapper Method"] = accuracy_score_rf2

3. Use any embedded methood to select the best features

In [18]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
embeddedX = df.drop('MedHouseVal', axis=1) # Features
embeddedy = df['MedHouseVal'] # Target variable
X_train, X_test, y_train, y_test = train_test_split(embeddedX, embeddedy, random_state=0,test_size=0.3)

lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)
score = r2_score(y_test, y_pred)

selected_features = selector.get_support(indices=True)
print(f"Selected features using Lasso: {selected_features}")
print(f"Score: {score}")

score_list["Embedded Method"] = score


Selected features using Lasso: [0 1 2 5 6]
Score: 0.589647943965883


In [19]:
score_list = list(score_list.items())
for alg,score in score_list:
    print(f"{alg} Score is {str(score)[:4]} ")

W/O Feature Selection Score is 0.60 
Filter Method Score is 0.54 
Wrapper Method Score is 0.54 
Embedded Method Score is 0.58 
